In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import image_dataset_from_directory

In [ ]:
# Import the function

# Veri setlerini içeri aktarma (mevcute kodunuzdan alındı)
train_ds = image_dataset_from_directory(
    "/content/cats_and_dogs_filtered/train",
    image_size=(256, 256),
    batch_size=32,
    label_mode="binary"
)

val_ds = image_dataset_from_directory(
    "/content/cats_and_dogs_filtered/validation",
    image_size=(256, 256),
    batch_size=32,
    label_mode="binary"
)

# --- Normalizasyon eklemesi ---
# Bu işlem, piksel değerlerini 0-255'ten 0-1 aralığına ölçekleyecektir.
normalization_layer = tf.keras.layers.Rescaling(1./255)

normalized_train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Artık eğitim ve doğrulama için normalized_train_ds ve normalized_val_ds kullanacağız

NotFoundError: Could not find directory /content/cats_and_dogs_filtered/train

In [ ]:


# Data Augmentation Katmanları
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"), # Rastgele yatay çevirme
    tf.keras.layers.RandomRotation(0.1),     # Rastgele dönüş (10 dereceye kadar)
    tf.keras.layers.RandomZoom(0.1),         # Rastgele yakınlaştırma
    tf.keras.layers.RandomContrast(0.1)      # Rastgele kontrast değişimi
])

# Normalize edilmiş eğitim veri setine veri büyütmeyi uygula
augmented_train_ds = normalized_train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

# Doğrulama veri setine veri büyütme uygulanmaz, sadece normalizasyonlu hali kullanılır
# augmented_train_ds ile eğiteceğiz, normalized_val_ds ile valide edeceğiz.

NameError: name 'normalized_train_ds' is not defined

In [ ]:
# Yeni CNN modeli oluşturma (base model)
model = Sequential()

# Feature extraction: CONV => RELU => CONV => RELU => POOL => DROPOUT
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(256, 256, 3)))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Bu aşamada küçük bir dropout eklenebilir, ancak genellikle daha sonraki katmanlar için tercih edilir.

# Feature extraction: CONV => RELU => CONV => RELU => POOL => DROPOUT
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) # Buradaki dropout iyi bir başlangıç

# Classification: FLATTEN, DENSE, RELU, DROPOUT, DENSE (OUTPUT LAYER)
model.add(Flatten()) # Vektör oluşur
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5)) # !!! Buraya yeni bir Dropout katmanı ekleyin !!!
model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_71 (Conv2D)              │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_72 (Conv2D)              │ (None, 252, 252, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_37 (MaxPooling2D) │ (None, 126, 126, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_73 (Conv2D)              │ (None, 124, 124, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_74 (Conv2D)              │ (None, 122, 122, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_38 (MaxPooling2D) │ (None, 61, 61, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_75 (Conv2D)              │ (None, 59, 59, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_76 (Conv2D)              │ (None, 57, 57, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_39 (MaxPooling2D) │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 512)            │    51,380,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,668,257 (197.10 MB)

 Trainable params: 51,668,257 (197.10 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.save('model_save.h5')

In [ ]:
from tensorflow.keras.optimizers import Adam # Adam optimizer'ı import et

# Model derlemesi
model.compile(
    optimizer=Adam(learning_rate=0.0001), # !!! Öğrenme oranını düşürdük !!!
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
# Model eğitimi
history = model.fit(
    augmented_train_ds, # !!! Burada artırılmış veri setini kullanın !!!
    epochs=25,
    validation_data=normalized_val_ds # !!! Burada normalize edilmiş doğrulama setini kullanın !!!
)

NameError: name 'model' is not defined

In [ ]:

# Adım 1: Görüntüyü yükle (boyutu modele göre ayarla)
img_path = "/content/köpek2.jpeg"
img = image.load_img(img_path, target_size=(256, 256))  # ya da modelin beklediği boyut

# Adım 2: Görüntüyü array'e çevir
img_array = image.img_to_array(img)

# Adım 3: Normalize et
img_array = img_array / 255.0

# Adım 4: Batch boyutu ekle (1, 128, 128, 3)
img_array = np.expand_dims(img_array, axis=0)

# Adım 5: Tahmin yap
prediction = model.predict(img_array)

# Adım 6: Sonucu yorumla
if prediction[0][0] > 0.5:
    print(f"Köpek (%{prediction[0][0]*100:.2f})")
else:
    print(f"kedi (%{(1 - prediction[0][0])*100:.2f})")



NameError: name 'model' is not defined